### Collect event statistics for each turker

In [1]:
import psycopg2 as pg
import numpy as np
import pandas as pd
import datetime
import logging
import math

# connect to turker database
connection = pg.connect(
    database="sidewalk_turker",
    user="sidewalk",
    password="sidewalk",
    host="localhost",
    port="5432")

In [2]:
turk_ids = pd.read_csv('../../data/interim/turk-ids.csv')
turk_ids.head()

,condition_id,worker_id
0,70,A3PPRVK6XK6GP5
1,72,A1TNQU9L5L62PL
2,74,A20BMZQJS92QY2
3,75,A3AUDX0SE41K23
4,76,A2DU8E2MAIGII6


In [3]:
event_types = pd.read_csv('../../data/interim/event-types.csv')
event_types = event_types['event_type'].tolist()
event_types[:5]

['Click_LabelDelete',
 'Click_ModeSwitch_CurbRamp',
 'Click_ModeSwitch_NoCurbRamp',
 'Click_ModeSwitch_NoSidewalk',
 'Click_ModeSwitch_Obstacle']

In [4]:
feature_names = pd.read_csv('../../data/interim/feature-names.csv')
feature_names = feature_names['feature'].tolist()
feature_names[:5]

['Click_LabelDelete_per_pan_mean',
 'Click_LabelDelete_per_pan_std',
 'Click_LabelDelete_total',
 'Click_ModeSwitch_CurbRamp_per_pan_mean',
 'Click_ModeSwitch_CurbRamp_per_pan_std']

In [5]:
'''
Sequence of queries to retrieve interactions for a user
'''

def get_assignment_id(condition_id, turker_id):
    return pd.read_sql(
    '''
        SELECT amt_assignment_id
        FROM amt_assignment
        WHERE condition_id='%(condition_id)s' AND turker_id='%(turker_id)s'
    ''' % locals(), connection)

def get_route_ids(condition_id, turker_id):
    return pd.read_sql(
    '''
        SELECT DISTINCT route_id
        FROM amt_assignment
        WHERE condition_id='%(condition_id)s' AND turker_id='%(turker_id)s'
    ''' % locals(), connection)

def get_street_edges(route_id):
    return pd.read_sql(
    '''
        SELECT current_street_edge_id
        FROM route_street
        WHERE route_id='%(route_id)s'
    ''' % locals(), connection)

def get_audit_tasks(assignment_id, street_edge):
    return pd.read_sql(
    '''
        SELECT audit_task_id
        FROM audit_task
        WHERE amt_assignment_id='%(assignment_id)s' AND street_edge_id='%(street_edge)s'
    ''' % locals(), connection)

def get_audit_interactions(audit_task_id):
    return pd.read_sql(
    '''
        SELECT *
        FROM audit_task_interaction
        WHERE audit_task_id='%(audit_task_id)s'
    ''' % locals(), connection)

In [6]:
'''
Find the total, mean, and std of each event type
'''

def get_action_counts(audit_inter):
    features = {}
    
    for action in event_types:
        a = audit_inter[audit_inter['action'] == action]
        g = a.groupby('gsv_panorama_id').size()
        
        total = len(a)
        mean = g.mean()
        std = g.std()
        
        # if there are too few elements to compute
        if math.isnan(mean):
            mean = 0
            
        if math.isnan(std):
            std = 0
            
        features[action + '_total'] = total
        features[action + '_per_pan_mean'] = mean
        features[action + '_per_pan_std'] = std
    
    return features

In [7]:
'''
Find all features for a turker
'''

def get_features(condition_id, worker_id):
    all_audit_interactions = None
    logging.debug('Condition_id, worker_id: ' + str(condition_id) + ', ' + str(worker_id))
    
    feature_df = pd.DataFrame(columns=(['condition_id', 'worker_id'] + feature_names))
    
    assignment_id = get_assignment_id(condition_id, worker_id)
    if len(assignment_id) == 0:
        logging.warning('SKIPPING condition_id, worker_id: ' + str(condition_id) + ', ' + str(worker_id))
        return
    
    assignment_ids = assignment_id['amt_assignment_id']
    routes = get_route_ids(condition_id, worker_id)
    
    for assignment_id in assignment_ids:
        logging.debug('Assignment id: ' + str(assignment_id))

        for i, route in routes[:3].iterrows():
            route_number = route['route_id']
            street_edges = get_street_edges(route_number)

            logging.debug('  Route_id: ' + str(route_number))

            for j, street_edge_series in street_edges.iterrows():
                street_edge = street_edge_series['current_street_edge_id']
                audit_tasks = get_audit_tasks(assignment_id, street_edge)

                logging.debug('    Street edge: ' + str(street_edge))
                logging.debug('    Num audit_tasks: ' + str(len(audit_tasks)))

                for k, audit_task in audit_tasks.iterrows():
                    audit_task_id = audit_task['audit_task_id']
                    audit_interactions = get_audit_interactions(audit_task_id)

                    logging.debug('      Audit task id: ' + str(audit_task_id))
                    logging.debug('      Num interactions: ' + str(len(audit_interactions)))

                    if all_audit_interactions is None:
                        all_audit_interactions = audit_interactions
                    else:
                        all_audit_interactions = all_audit_interactions.append(audit_interactions)
                
    if all_audit_interactions is None:
        logging.warning('SKIPPING because no iteraction: ' + str(condition_id) + ', ' + str(worker_id))
        return
    action_counts = get_action_counts(all_audit_interactions)
    
    feature_list = [action_counts[feature] for feature in feature_names]
    feature_df.loc[len(feature_df)] = [condition_id, worker_id] + feature_list
    
    return feature_df

In [8]:
'''
Collect features for each turker (long operation)
'''

features = None

for i, user in turk_ids.iterrows():
    
    f = get_features(user['condition_id'], user['worker_id'])
    if features is None:
        features = f
    else:
        features = features.append(f)

features.reset_index(inplace=True)

In [9]:
features.head()

,index,condition_id,worker_id,Click_LabelDelete_per_pan_mean,Click_LabelDelete_per_pan_std,Click_LabelDelete_total,Click_ModeSwitch_CurbRamp_per_pan_mean,Click_ModeSwitch_CurbRamp_per_pan_std,Click_ModeSwitch_CurbRamp_total,Click_ModeSwitch_NoCurbRamp_per_pan_mean,...,ViewControl_DoubleClick_total,ViewControl_MouseDown_per_pan_mean,ViewControl_MouseDown_per_pan_std,ViewControl_MouseDown_total,ViewControl_MouseUp_per_pan_mean,ViewControl_MouseUp_per_pan_std,ViewControl_MouseUp_total,WalkTowards_per_pan_mean,WalkTowards_per_pan_std,WalkTowards_total
0,0,70,A3PPRVK6XK6GP5,2.5,2.12132,5,3.38889,2.22655,61,2,...,130,7.235294,8.381736,615,6.917647,7.850101,588,0,0,0
1,0,72,A1TNQU9L5L62PL,0,0,0,2.21951,2.603,91,1.44737,...,23,5.656489,6.420845,741,5.389313,6.039068,706,0,0,0
2,0,74,A20BMZQJS92QY2,1.18182,0.40452,13,2.35897,1.76944,92,1.84615,...,1,8.060606,5.626759,1064,7.878788,5.523068,1040,0,0,0
3,0,75,A3AUDX0SE41K23,1,0,4,4.92,2.5318,123,1,...,13,8.504274,9.576933,995,7.709402,8.266113,902,0,0,0
4,0,76,A2DU8E2MAIGII6,1.66667,0.57735,5,3.66667,2.3094,11,1,...,224,7.772727,7.697928,684,7.511364,7.345331,661,0,0,0


In [10]:
len(features)

264

In [11]:
len(turk_ids)

264

In [12]:
del features['index']
features.to_csv('../../data/interim/collected/turk-features.csv', index=False)